In [3]:
import pandas as pd
from collections import Counter
import ast

In [4]:
# Load dataset
df = pd.read_csv("stockist_data.csv")

In [5]:
# Define relevant product columns
product_columns = [
    'AIS(Air Insulated Switchgear)', 'RMU(Ring Main Unit)', 'PSS(Compact Sub-Stations)',
    'VCU(Vacuum Contactor Units)', 'E-House', 'VCB(Vacuum Circuit Breaker)', 'ACB(Air Circuit Breaker)',
    'MCCB(Moduled Case Circuit Breaker)', 'SDF(Switch Disconnectors)', 'BBT(Busbar Trunking)',
    'Modular Switches', 'Starter', 'Controller', 'Solar Solutions', 'Pump Starter and Controller'
]
existing_product_columns = [col for col in product_columns if col in df.columns]


In [6]:
# Initialize tracking variables
product_scheme_data = []

In [7]:
# Process each product separately
for product in existing_product_columns:
    product_df = df[df[product] == 1]  # Filter stockists selling this product
    
    if product_df.empty:
        continue  # Skip products without stockists

    # Extract relevant columns
    partner_ids = product_df['Partner_id'].dropna().astype(str).unique()

    # Consider only Sales Quantity for scheme selection
    scheme_data = product_df[['Scheme_Type', 'Sales_Quantity_Last_Period']].dropna()
    
    # Compute scheme impact (Weighted by Sales Quantity) for each product separately
    scheme_growth = Counter()
    
    for _, row in scheme_data.iterrows():
        schemes = row['Scheme_Type'].split(', ') if isinstance(row['Scheme_Type'], str) else []
        for scheme in schemes:
            scheme_growth[scheme] += row['Sales_Quantity_Last_Period']

    # Select top 3 schemes for this specific product
    top_schemes = [s[0] for s in scheme_growth.most_common(3)]
    while len(top_schemes) < 3:
        top_schemes.append("No Scheme Available")

    # Store the processed data
    product_scheme_data.append({
        'Product_id': product,
        'Partner_id': ', '.join(partner_ids),
        'Scheme_1': top_schemes[0],
        'Scheme_2': top_schemes[1],
        'Scheme_3': top_schemes[2]
    })


In [8]:
# Convert results into a DataFrame
final_df = pd.DataFrame(product_scheme_data)

# Save intermediate results
final_scheme_mapping_path = "Optimized_Product_Partner_Scheme_Mapping.csv"
final_df.to_csv(final_scheme_mapping_path, index=False) 